## Imports 

In [1]:
import numpy as np
import pandas as pd
from math import log
from itertools import combinations
from sklearn.metrics import f1_score

## Utility Functions

In [2]:
"""
Takes a dataframe and an attribute as input. Computes the entropy of splitting
the dataset on the values of the attribute.
"""
def gain(data,D):
    m = len(data)
    if m <= 1: return 0
    values, counts = np.unique(data[D],return_counts=True)
    probs = counts / m # probabilities of D's values
    ent = sum([prob * entropy(data[data[D]==val]) for prob,val in zip(probs,values)])
    return entropy(data) - ent

"""
Takes a dataframe as input. It computes the entropy of a target varaible specified globally
"""
def entropy(data):
    m = len(data)
    if m <= 1: return 0
    value, counts = np.unique(data[target], return_counts=True)
    probs = counts / m # probabilities of y's values 
    n_classes = np.count_nonzero(probs)
    if n_classes <= 1: return 0
    return -sum([p * log(p,2) for p in probs])

"""
Takes a dataframe and attribute as input. Returns the normalized information gain
of splitting the data on the attribute.
"""
def gain_ratio(data,D):
    values,counts = np.unique(data[D],return_counts=True)
    get_ratio = lambda x: (x / len(data)) * log((x / len(data)),2)
    intrinsic_value = -sum([get_ratio(x) for x in counts])
    return 0 if intrinsic_value == 0 else gain(data,D) / intrinsic_value

## Decision Tree Learn and Predict Algorithms

In [3]:
"""
This algorithm recursively learns a decision tree to fit the input data.
ID3 is used when the Information Gain is passed in as the importance function
and C4.5 is used when the Gain Ratio is passed in. 

Input:
data - a pandas dataframe of data to utilize
target - the string that is the name of the target column in the data dataframe
attributes - the string list of data's column/header/feature names (minus the target)
level - passed in as 0. Increments upon recursion to help debugging
importance - a function which takes the data and an attribute in the dataframe as input/
        - if 'gain' is used, this algorithm becomes Quinlan's ID3 algorithm
        - if 'gain_ratio' is used, this algorithm is more like Quinlan's C4.5 algorithm
debug - a boolean, where True triggeres print methods to print what the algorithm is doing

Output: returns a nested python dictionary corresponding to the learned tree
"""
def dtree_learn(data,target,attributes,level,importance,debug=False):
    if data[target].all(): 
        child = {'label':'leaf','value':1}
        if debug: print ("{}All data is pos, returning pos leaf".format("\t"*level))
    elif data[target].any():
        if len(attributes) == 0: 
            val = 1 if data[target].mode().values[0] else 0
            child = {'label':'leaf','value':val}
            if debug: print ("{}no more attributes, returning leaf with majority label".format("\t"*level))
        else:
            best = attributes[np.argmax([importance(data,attr) for attr in attributes])]
            child = {'label':best,'children':[]}
            if debug: print("{}Created {} node".format("\t"*level,best))
            for val in data[best].unique():
                new_child = {'label':'node','parent attr':best,'parent val':val,'children':[]}
                new_data = data[data[best]==val]
                if len(new_data) == 0: 
                    val = 1 if data[target].mode().values[0] else 0
                    new_child['children'].append({'label':'leaf','value':val}) 
                    if debug: print("{}no more data, returning leaf with majority label".format("\t"*level))
                else: 
                    if debug: print ("{}recursing to next ID3 iteration for {}'s value {}".format("\t"*level,best,val))
                    new_child.update(dtree_learn(new_data,target,attributes.drop(best),level+1,importance,debug)) 
                child['children'].append(new_child)
    else: 
        child = {'label':'leaf','value':0}
        if debug: print("{}all data is neg, returning neg leaf".format("\t"*level))
    return child

"""
This predict function traverses a decision tree according to the features of
an input example until it reaches a leaf node. When a leaf node is reached, it
returns the label on the leaf, which is the predicted label for the example

Input:
dtree - a decision tree in the form of nested Python dictionaries
example - an input example to classify

Ouput: returns 0 or 1
"""
def dtree_predict(dtree,example):
    if dtree['label'] != 'leaf':
        for child in dtree['children']:
            if child['parent val'] == example[child['parent attr']]:
                return dtree_predict(child,example)
    else: return dtree['value']

## Load and Preprocess the Data

##### Loading the Cross-Validation training and testing datasets

In [4]:
# load the cross-validation train datasets and the testing dataset
cv_train_files = ["Mushroom/training_a{}.data".format(x) for x in ['a','b','c','d','e','f','g','h','i','j']]
cv_train_data = [pd.read_csv(tf) for tf in cv_train_files]
test_data = pd.read_csv("Mushroom/testing.data")

# convert target vectors in all data to binary vectors
target, pos_val = 'edible','e' # denoting 'e' and 'p' as the positive and negative classes, respectively
for td in cv_train_data: td[target] = td[target].apply(lambda x: 1 if x == pos_val else 0)
test_data[target] = test_data[target].apply(lambda x: 1 if x == pos_val else 0)

## ID3 with 10-fold cross validation

##### Training and Cross-Validation

In [5]:
# perform 10-fold cross validation
id3_trees_n_scores = []
for i,valid_data in enumerate(cv_train_data):
    merged_train_data = pd.concat(cv_train_data[:i]+cv_train_data[i+1:])
    dtree = dtree_learn(merged_train_data,target,merged_train_data.columns.drop(target),0,gain) # gain used: ID3
    predictions = valid_data.apply(lambda x: dtree_predict(dtree,x),axis=1)
    id3_trees_n_scores.append((dtree,f1_score(valid_data[target],predictions)))
print ("ID3 scores on validation data: {}".format([score for _,score in id3_trees_n_scores]))

ID3 scores on validation data: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


##### Testing

In [6]:
# all models have equal cross-validation performance, so selecting the first will do
best_id3_tree = id3_trees_n_scores[0][0]

# now testing the best model on the test set
predictions = test_data.apply(lambda x: dtree_predict(best_id3_tree,x),axis=1)
print ("Best ID3 tree's F1-score: {}".format(f1_score(test_data[target],predictions)))

Best ID3 tree's F1-score: 1.0


## C4.5 with 10-fold cross-validation

##### Training and Cross-Validation

In [7]:
# perform 10-fold cross-validation
c45_trees_n_scores = []
for i,valid_data in enumerate(cv_train_data):
    merged_train_data = pd.concat(cv_train_data[:i]+cv_train_data[i+1:])
    dtree = dtree_learn(merged_train_data,target,merged_train_data.columns.drop(target),0,gain_ratio) #gain ratio used: C4.5
    predictions = valid_data.apply(lambda x: dtree_predict(dtree,x),axis=1)
    c45_trees_n_scores.append((dtree,f1_score(valid_data[target],predictions)))
print ("C4.5 scores on validation data: {}".format([score for _,score in c45_trees_n_scores]))

C4.5 scores on validation data: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


##### Testing

In [8]:
# all models have equal cross-validation performance, so selecting the first will do
best_c45_tree = c45_trees_n_scores[0][0]

# now testing the best model on the test set
predictions = test_data.apply(lambda x: dtree_predict(best_c45_tree,x),axis=1)
print ("Best C4.5 tree's F1-score: {}".format(f1_score(test_data[target],predictions)))

Best C4.5 tree's F1-score: 1.0
